# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.1734, -0.0765, -0.2538, -0.2185,  0.2323, -0.1510,  0.0465,  0.0610,
         -0.0841,  0.0671],
        [ 0.1561, -0.0076, -0.1620, -0.2349,  0.2478, -0.1030, -0.0291,  0.0409,
         -0.0872,  0.1476]], grad_fn=<AddmmBackward>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[ 0.1473, -0.0081, -0.1485,  0.0613,  0.1064,  0.0512,  0.0210, -0.2761,
         -0.0448,  0.0043],
        [-0.0538,  0.0762, -0.0525, -0.0285,  0.0038,  0.1172,  0.1184, -0.1988,
          0.0881, -0.0972]], grad_fn=<AddmmBackward>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1614, -0.1057, -0.0085,  0.1822,  0.0858,  0.1759, -0.0407, -0.1523,
         -0.0376, -0.0554],
        [-0.1905, -0.0571, -0.0363,  0.2945,  0.1853,  0.1672, -0.2142, -0.1598,
         -0.0810,  0.0916]], grad_fn=<AddmmBackward>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0772, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1996, grad_fn=<SumBackward0>)